In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Getting the data

In [ ]:
df=pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum()  #we will not convert values of -1 to none ,just leave them

In [ ]:
df[df['winPlacePerc'].isna()]

In [ ]:
df=df.drop(2744604)            #dropping this nan
df=df.reset_index(drop=True)

In [ ]:
df.matchId.value_counts()        #it seems the data have all players os some matches 
                                 #but actually I didn't use it

In [ ]:
df[df.killPoints==0][df.rankPoints!=-1].shape    #getting the shape of 0 kill points and no rank

In [ ]:
df[df.killPoints==0].shape                   #it seems all 0 kill points has no rank

In [ ]:
df.describe()          #getting the statistical analysis of the data 

In [ ]:
df.matchType.value_counts() 

# Visualizing the data and getting insights

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(13,7))
sns.boxplot(x='matchType',y='winPlacePerc',data=df)     
plt.xticks(rotation=300)
plt.show()


#it seems the type of match isn't playing a big role in winning

In [ ]:
sns.lmplot(x='winPlacePerc',y='kills',data=df,fit_reg=False)
plt.show()

#A lot wins without even one kill, may be they are cheaters or have a great team beside them
#but in general more kills more wins

In [ ]:
sns.lmplot(x='winPlacePerc',y='winPoints',data=df,fit_reg=False)
plt.show()

#win points don't give you more probabilty to win ever

In [ ]:
sns.lmplot(x='winPlacePerc',y='killPoints',data=df,fit_reg=False)
plt.show()

#neither kill points

In [ ]:
sns.lmplot(x='winPlacePerc',y='rankPoints',data=df,fit_reg=False)
plt.show()

#and rank is trash 

In [ ]:
sns.lmplot(x='winPlacePerc',y='longestKill',data=df,fit_reg=False)
plt.show()

#okay it has a little tiny correlation but a lot of noise 

In [ ]:
sns.lmplot(x='winPlacePerc',y='damageDealt',data=df)
plt.show()

#some winners didn't even make a scratch ,but you can see a slight correlation though

In [ ]:
sns.lmplot(x='winPlacePerc',y='walkDistance',data=df)
plt.show()

#winners without even walk ,hackers

In [ ]:
df[df['walkDistance']==0][df['winPlacePerc']==1].describe()

#and a lot of cheaters here weapons and kills without a single move

In [ ]:
sns.lmplot(x='winPlacePerc',y='swimDistance',data=df,fit_reg=False)
plt.show()

#not interesting 

## Converting some numerical features into categories

In [ ]:
weapons_num=pd.cut(df['weaponsAcquired'],[0,2,6,12,25,50,100,200,300],
                   labels=['0-2','2-6','6-12','12-25','25-50','50-100','100-200','+200'])

#making the number of weapons acquired in categories so we can visualize it properly

In [ ]:
plt.figure(figsize=(12,7))
sns.boxplot(x=weapons_num,y='winPlacePerc',data=df)
plt.show()

#okay having weapons from 6 and up to 25 makes your probability to win a bit more
#it's the proper number beacuse more than 100 hunderd are likely cheatings

In [ ]:
del weapons_num

In [ ]:
kills=pd.cut(df['kills'],[1,10,20,40,60,80],labels=['1-10','10-20','20-40','40-60','+60'])
#make kills in categories

In [ ]:
plt.figure(figsize=(13,6))
sns.boxplot(x=kills,y=df['winPlacePerc'])
plt.show()

In [ ]:
df.headshotKills.value_counts()

#64 headshot !

In [ ]:
head_shot=pd.cut(df['headshotKills'],[0,1,2,3,4,7,10,16,19,35,64],
                 labels=['0','1','2','3','4-7','7-10','10-16','16-19','19-35','35-64'])

#making head shots in categories

In [ ]:
plt.figure(figsize=(13,6))
sns.boxplot(x=head_shot,y=df['winPlacePerc'])

#if you get more than 5 head shots you are likely more to win 

# Adding new features

In [ ]:
groups=df.groupby('groupId').sum()

#we will make some features that take all the group in consideration

In [ ]:
groups=groups.reset_index()

In [ ]:
groups.head()

In [ ]:
add_col=groups.loc[:,['groupId','kills','revives','weaponsAcquired','teamKills']]

#These are the new columns

In [ ]:
add_col=add_col.rename(columns={'kills':'groupKills','revives':'groupRevives',
                                'weaponsAcquired':'groupWeapons','teamKills':'groupOwnKills'})

#let's give them nice names 

In [ ]:
df2=df.merge(add_col,how='left',on=['groupId'])
#and we merge them with the data

In [ ]:
df2.head()

In [ ]:
del df 
del groups
del add_col
#then get rid of them 

In [ ]:
df2.loc[:,['groupKills','groupRevives','groupWeapons','groupOwnKills']].describe()

## Let's visualize these new features 

In [ ]:
gkills=pd.cut(df2['groupKills'],[1,10,20,40,60,80,200],
              labels=['1-10','10-20','20-40','40-60','60-80','+80'])

In [ ]:
plt.figure(figsize=(13,6))
sns.boxplot(x=gkills,y=df2['winPlacePerc'])
plt.show()

In [ ]:
gweapons_num=pd.cut(df2['groupWeapons'],[0,2,6,12,25,50,100,200,300,700],
                labels=['0-2','2-6','6-12','12-25','25-50','50-100','100-200','200-300','+300'])

In [ ]:
plt.figure(figsize=(13,6))
sns.boxplot(x=gweapons_num,y=df2['winPlacePerc'])
plt.show()

#Your group should have from 12 to 25 to be close to win 
#because the game is not about collecting weapons

In [ ]:
grevives=pd.cut(df2['groupRevives'],[0,2,4,8,15,30,50],
                labels=['0-2','2-4','4-8','8-15','15-30','+30'])

In [ ]:
plt.figure(figsize=(13,6))
sns.boxplot(x=grevives,y=df2['winPlacePerc'])
plt.show()

> # So to make your win more certain, you should have minimum 6 weapons ,10 kills,5 headshots,your group make about 4 revives and run a lot if you are not a hacker anyway

# Preparing the data for the model

In [ ]:
df2.head()

In [ ]:
df2['matchType']=df2.matchType.astype('category') #make it categorical to encode

In [ ]:
df2['matchtype']=df2['matchType'].cat.codes  #encoding

In [ ]:
df2.head()

In [ ]:
pubg=df2.drop(columns=['Id','groupId','matchId','matchType'])  

In [ ]:
del df2

In [ ]:
pubg=pubg.drop(columns=['killPoints','matchDuration','maxPlace','numGroups','rankPoints'
                        ,'winPoints','groupOwnKills','teamKills','vehicleDestroys'])

#dropping the unnecessary columns

## Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x=pubg.drop(columns=['winPlacePerc'])
y=pubg['winPlacePerc']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
del x
del y

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(pubg.corr(),annot=True)

In [ ]:
corr_matrix = pubg.corr()
corr_matrix["winPlacePerc"].sort_values(ascending=False)

#It seems that some of our new features are good correlated 

# Linear Regression model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = lin_reg.predict(x_test)
lin_mse = mean_squared_error(y_test, y_pred)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
del lin_reg
del lin_mse
del lin_rmse

# Decision Tree model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(x_train, y_train)

In [ ]:
y_pred = tree_reg.predict(x_test)
tree_mse = mean_squared_error(y_test, y_pred)
tree_rmse = np.sqrt(tree_mse)
tree_rmse
#great

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, x_train, y_train,
                         scoring="neg_mean_squared_error", cv=5)

In [ ]:
-scores

In [ ]:
del scores

# Model tunning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'max_features': [6, 8,None]},
  ]

grid_search = GridSearchCV(tree_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(x_train, y_train)

In [ ]:
grid_search.best_params_
#I knew it

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(pubg.drop(columns=['winPlacePerc']), pubg['winPlacePerc'])

#Fitting the model with all the data and it's ready to go

## Handling the test set

In [ ]:
df_test=pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
groups=df_test.groupby('groupId').sum()
groups=groups.reset_index()

In [ ]:
add_col=groups.loc[:,['groupId','kills','revives','weaponsAcquired','teamKills']]
add_col=add_col.rename(columns={'kills':'groupKills','revives':'groupRevives','weaponsAcquired':'groupWeapons','teamKills':'groupOwnKills'})
df2_test=df_test.merge(add_col,how='left',on=['groupId'])

In [ ]:
df2_test.head()

In [ ]:
df2_test['matchType']=df2_test.matchType.astype('category')
df2_test['matchtype']=df2_test['matchType'].cat.codes

In [ ]:
final=df2_test.drop(columns=['Id','groupId','matchId','matchType'])
final=final.drop(columns=['killPoints','matchDuration','maxPlace','numGroups',
                          'rankPoints','winPoints','groupOwnKills','teamKills','vehicleDestroys'])

In [ ]:
Id=df2_test['Id']  #for submission

In [ ]:
winPlacePerc = tree_reg.predict(final)  #our prediction

In [ ]:
pred =pd.concat([Id,pd.Series(winPlacePerc)],axis=1)

In [ ]:
pred=pred.rename(columns={0:'winPlacePerc'})

In [ ]:
pred.to_csv('submission.csv',index=False) 